In [2]:
import numpy as np
import random
from numpy import array, where, eye, linspace, matrix, hstack
from scipy import integrate
from scipy.stats import lognorm,gamma
from numpy import array, where
from numpy.random import seed, shuffle
import pandas as pd


In [3]:
def dX_dt_template(C):
    return lambda X, t=0: X*array(matrix(C)*matrix(hstack((array([1]),X))).T).reshape(len(X))

def lokta_volterra(dX_dt, X0, lb, ub, ts):
    t = linspace(lb, ub,  ts) #timesteps to eval lv at
    X = integrate.odeint(dX_dt, X0, t)
    return X.T #transpose to make otusXsamples

def model1_otu(df_and_params, samples):
    """Return an otu vector drawn from df given params and of length samples."""
    return df_and_params[0].rvs(*df_and_params[1:],size=samples)

def model1_table(dfs_and_params, samples):
    return np.array([model1_otu(i,samples) for i in dfs_and_params])

def alter_table(data, as_abund=True, as_int=False, sparsity=.8):
    """Change table to RA, to int, and/or add sparsity."""
    res = data
    if not as_abund:
        res = res/res.sum(0)
    if as_int:
        res = res.round(0)
    if sparsity:
        r,c = where(data==data)
        q = list(zip(r,c))
        shuffle(q)
        for i in range(int(len(q)*sparsity)):
            res[q[i]] = 0.
    return res

In [4]:
random.seed(30968340693)
two_species_interaction_matrices = [\
array([[1., 0, -.1],
[-1.5, 0.075, 0]]),
array([[1., 0, -.1],
[-0.5, 0.075, 0]]),
array([[1, 0, -.1],
[-10, 0.075, 0]]),
array([[10, 0, -.1],
[-10, 0.075, 0]]),
array([[-1, 0, -.1],
[-1.5, 0.075, 0]]),
array([[-0.1, 0, -.1],
[-1.5, 0.075, 0]]),
array([[0, 0, -.1],
[-1.5, 0.075, 0]]),
array([[0, 0, -.1],
[0, 0.075, 0]]),
array([[1., 0, -1],
[-1.5, 1, 0]]),
array([[4, -2, -2],
[1, 1, -2]])]

six_species_interaction_matrices = [\
array([[2, 0, -2, 0, 0, 0, 0],
[1, 2, -1, -2, 0, 0, 0],
[4, 0, 2, 0, -2, -2, -2],
[-1, 0, 0, 2, 0, -1, 0],
[-3, 0, 0, 2, 1, 0, 0],
[-1, 0, 0, 2, 0, 0, -1]]),
array([[2,-10,-2,0,0,0,0],
[1,10,-1,-2,0,0,0],
[4,10,2,0,-2,-2,-2],
[-1,-10,0,2,0,-1,0],
[-3,-10,0,2,1,0,0],
[-10,-10,0,2,0,0,-1]]),
array([[10,-10,-2,0,0,0,0],
[1,10,-1,-2,0,0,0],
[4,10,2,0,-2,-2,-2],
[-1,-10,0,2,0,-1,0],
[-3,-10,0,2,1,0,0],
[-1,-10,0,2,0,0,-1]])]

In [5]:
# two_species_otu_table
two_species_otu_table = []
for C in two_species_interaction_matrices:
    f = dX_dt_template(C)
    Y = lokta_volterra(f, array([10]*C.shape[0]), 0, 20, 10000)
    two_species_otu_table.extend([Y[0],Y[1]])




In [6]:

# shape is 20 x 10000
two_species_otu_table = np.vstack(two_species_otu_table)

# make 5 tables:
# 1. relative abundance table with pts taken at equal intervals
# 2. counts table with pts taken at equal intervals (same pts as 1)
# 3. relative abundance table with pts taken at random indices
# 4. counts table with pts taken at random intervals (same pts as 3)
# 5. table with 60 percent sparsity 

# must add 80 completely random OTUs to confound + pad the table

# generate 40 otus from lognorm distribution 2,0,1 
dfs_and_params = [[lognorm, 2, 0]]*40
otus_lognorm_2_0 = model1_table(dfs_and_params, 50)
# generate 40 otus from gamma distribution 1,0,10
dfs_and_params = [[gamma, 1, 0, 10]]*40
otus_gamma_1_0_10 = model1_table(dfs_and_params, 50)



In [7]:

evenly_sampled_indices = np.arange(50)*200
random_indices = np.arange(10000)
random.shuffle(random_indices)
random_indices = random_indices[:50]



In [8]:
lv_table_2sp_12_base = np.vstack([two_species_otu_table[:,evenly_sampled_indices], 
    otus_gamma_1_0_10, otus_lognorm_2_0])


In [9]:
lv_table_2sp_1 = (lv_table_2sp_12_base/lv_table_2sp_12_base.sum(0))
lv_table_2sp_2 = 100*lv_table_2sp_12_base.round(0)


In [10]:
lv_table_2sp_34_base = np.vstack([two_species_otu_table[:,random_indices], 
    otus_gamma_1_0_10, otus_lognorm_2_0])


In [11]:
lv_table_2sp_3 = (lv_table_2sp_34_base/lv_table_2sp_34_base.sum(0))


In [12]:

lv_table_2sp_4 = 100*lv_table_2sp_34_base.round(0)


In [13]:
lv_table_2sp_5 = alter_table(lv_table_2sp_12_base, sparsity=.6).round(0)


In [14]:
# six_species_otu_tables
six_species_otu_tables = []
for C in six_species_interaction_matrices:
    f = dX_dt_template(C)
    Y = lokta_volterra(f, array([10]*C.shape[0]), 0, 20, 10000)
    six_species_otu_tables.append(Y)

six_species_otu_table = np.vstack(six_species_otu_tables)

# make 5 tables:
# 1. relative abundance table with pts taken at equal intervals
# 2. counts table with pts taken at equal intervals (same pts as 1)
# 3. relative abundance table with pts taken at random indices
# 4. counts table with pts taken at random intervals (same pts as 3)
# 5. table with 60 percent sparsity 

# must add 82 completely random OTUs to confound + pad the table

# generate 42 otus from lognorm distribution 1,0,1 
dfs_and_params = [[lognorm, 1, 0]]*42
otus_lognorm_1_0 = model1_table(dfs_and_params, 50)
# generate 42 otus from gamma distribution 1,0,10
dfs_and_params = [[gamma, 1, 0, 10]]*42
otus_gamma_1_0_10 = model1_table(dfs_and_params, 50)

evenly_sampled_indices = np.arange(50)*200
random_indices = np.arange(10000)
random.shuffle(random_indices)
random_indices = random_indices[:50]

lv_table_6sp_12_base = np.vstack([six_species_otu_table[:,evenly_sampled_indices], 
    otus_gamma_1_0_10, otus_lognorm_1_0])
lv_table_6sp_1 = (lv_table_6sp_12_base/lv_table_6sp_12_base.sum(0))
lv_table_6sp_2 = 100*lv_table_6sp_12_base.round(0)
lv_table_6sp_34_base = np.vstack([six_species_otu_table[:,random_indices], 
    otus_gamma_1_0_10, otus_lognorm_1_0])
lv_table_6sp_3 = (lv_table_6sp_34_base/lv_table_6sp_34_base.sum(0))
lv_table_6sp_4 = 100*lv_table_6sp_34_base.round(0)
lv_table_6sp_5 = alter_table(lv_table_6sp_12_base, sparsity=.6).round(0)




# small negatives exist in two of the lv tables because of floating point 
# error, need to resolve this.
lv_table_2sp_1_corr = where(lv_table_2sp_1>0, lv_table_2sp_1, 0)
lv_table_2sp_3_corr = where(lv_table_2sp_3>0, lv_table_2sp_3, 0)

all_tables = [lv_table_2sp_1_corr, lv_table_2sp_2, lv_table_2sp_3_corr, lv_table_2sp_4,
    lv_table_2sp_5, lv_table_6sp_1, lv_table_6sp_2, lv_table_6sp_3,
    lv_table_6sp_4, lv_table_6sp_5]


In [15]:
print(np.shape(lv_table_2sp_1_corr))
print(np.shape(lv_table_2sp_2))
print(np.shape(lv_table_2sp_3_corr))
print(np.shape(lv_table_2sp_4))
print(np.shape(lv_table_2sp_5))
print(np.shape(lv_table_6sp_1))
print(np.shape(lv_table_6sp_2))
print(np.shape(lv_table_6sp_3))
print(np.shape(lv_table_6sp_4))
print(np.shape(lv_table_6sp_5))

(100, 50)
(100, 50)
(100, 50)
(100, 50)
(100, 50)
(102, 50)
(102, 50)
(102, 50)
(102, 50)
(102, 50)


In [16]:
df1 = pd.DataFrame(lv_table_2sp_1_corr, index=['o'+str(i) for i in range(0, 100)], columns=['s'+str(i) for i in range(0, 50)])
df2 = pd.DataFrame(lv_table_2sp_2, index=['o'+str(i) for i in range(0, 100)], columns=['s'+str(i) for i in range(0, 50)])
df3 = pd.DataFrame(lv_table_2sp_3_corr, index=['o'+str(i) for i in range(0, 100)], columns=['s'+str(i) for i in range(0, 50)])
df4 = pd.DataFrame(lv_table_2sp_4, index=['o'+str(i) for i in range(0, 100)], columns=['s'+str(i) for i in range(0, 50)])
df5 = pd.DataFrame(lv_table_2sp_5, index=['o'+str(i) for i in range(0, 100)], columns=['s'+str(i) for i in range(0, 50)])
df6 = pd.DataFrame(lv_table_6sp_1, index=['o'+str(i) for i in range(0, 102)], columns=['s'+str(i) for i in range(0, 50)])
df7 = pd.DataFrame(lv_table_6sp_2, index=['o'+str(i) for i in range(0, 102)], columns=['s'+str(i) for i in range(0, 50)])
df8 = pd.DataFrame(lv_table_6sp_3, index=['o'+str(i) for i in range(0, 102)], columns=['s'+str(i) for i in range(0, 50)])
df9 = pd.DataFrame(lv_table_6sp_4, index=['o'+str(i) for i in range(0, 102)], columns=['s'+str(i) for i in range(0, 50)])
df10 = pd.DataFrame(lv_table_6sp_5, index=['o'+str(i) for i in range(0, 102)], columns=['s'+str(i) for i in range(0, 50)])

In [17]:
df1.to_csv('F:\\table\\Table Set 2.6.txt',sep='\t')
df2.to_csv('F:\\table\\Table Set 2.7.txt',sep='\t')
df3.to_csv('F:\\table\\Table Set 2.8.txt',sep='\t')
df4.to_csv('F:\\table\\Table Set 2.9.txt',sep='\t')
df5.to_csv('F:\\table\\Table Set 2.10.txt',sep='\t')
df6.to_csv('F:\\table\\Table Set 2.11.txt',sep='\t')
df7.to_csv('F:\\table\\Table Set 2.12.txt',sep='\t')
df8.to_csv('F:\\table\\Table Set 2.13.txt',sep='\t')
df9.to_csv('F:\\table\\Table Set 2.14.txt',sep='\t')
df10.to_csv('F:\\table\\Table Set 2.15.txt',sep='\t')